In [1]:
import csv
import numpy as np
import pandas as pd

In [2]:
#https://www.census.gov/data/datasets/time-series/demo/popest/2010s-state-total.html

In [3]:
# EDIT THIS TO CONTAIN ALL YEARS YOU WANT TO ANALYZE
years = ['2000', '2001', '2013', '2013', '2014', '2015', '2016', 
         '2017', '2018', '2019']

In [4]:
def combine(year):
    unemployment_csv = f'./data/{year}_unemployment.csv'
    education_csv = f'./data/education_{year}.csv'
    if int(year) < 2011:
        population_csv = f'./data/nst-est2000-alldata.csv'
    else:
        population_csv = f'./data/nst-est2019-alldata.csv'
    snaps_csv = f'./data/snaps_data_{year}.csv'
    
    unemployment_df = pd.read_csv(unemployment_csv)
    education_df = pd.read_csv(education_csv)
    pop_df = pd.read_csv(population_csv)
    #print(pop_df.head())
    #print(population_csv)

    pop_df = pop_df[["NAME", f"POPESTIMATE{year}"]].copy()
    snaps_df = pd.read_csv(snaps_csv)
    education_df = education_df.rename(columns={"State or Other Area": "State", f"{year} Actual": "Education Budget"})
    unemployment_df = unemployment_df.rename(columns={"state": "State", f"{year} rate": "Unemployment Rate"})
    #print(unemployment_df)
    pop_df = pop_df.rename(columns={"NAME": "State", f"POPESTIMATE{year}": "Population Estimate"})
    snaps_df = snaps_df.rename(columns = {"state": "State", "household_participants": "SNAPS household_participants",
                                   "individual_participants": "SNAPS individual_participants", "cost": "SNAPS cost",
                                   "cost_per_person": "SNAPS cost_per_person", "cost_per_household": "SNAPS cost_per_household"})
    
    df = pd.merge(education_df, unemployment_df, on='State')
    if 'rank' in df.columns:
        df = df.drop(columns='rank')
    df = pd.merge(df, pop_df, how='left', on='State')
    df = pd.merge(df, snaps_df, how='left', on='State')
    
    return df

In [5]:
for year in years:
    df = combine(year)
#     print(df.head())
    df.to_csv(f'./data/{year}-combined.csv', index=False)

df.head()

,State,Education Budget,Unemployment Rate,Population Estimate,SNAPS household_participants,SNAPS individual_participants,SNAPS cost,SNAPS cost_per_person,SNAPS cost_per_household
0,Alabama,2.775338e+09,3.2,4903185,315551,667043,1032064886,272.56,128.94
1,Alaska,4.005232e+08,5.6,731545,35493,78932,171091672,401.70,180.63
2,Arizona,5.143402e+09,4.8,7278717,345106,736883,1146712214,276.90,129.68
3,Arkansas,1.350409e+09,3.5,3017804,159231,354917,460333605,240.92,108.08
4,California,1.721222e+10,4.1,39512223,1803402,3529470,5977281299,276.20,141.13


In [9]:
for year in years:
    source = f'./data/{year}-combined.csv'
    ydf = pd.read_csv(source)
    ydf['Year'] = year
    if year != '2015':
        df = pd.concat([df, ydf], ignore_index=True)
    else:
        df = ydf.copy()



In [7]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
print(df)
df.to_csv('./data/data.csv', index=False)

             State  Education Budget  Unemployment Rate  Population Estimate  \
0          Alabama      2.729781e+09                6.1              4852347   
1           Alaska      4.152387e+08                6.5               737498   
2          Arizona      6.241051e+09                6.1              6829676   
3         Arkansas      1.334963e+09                5.2              2978048   
4       California      1.672784e+10                6.2             38918045   
..             ...               ...                ...                  ...   
246       Virginia      4.493670e+09                2.8              8535519   
247     Washington      2.471252e+09                4.2              7614893   
248  West Virginia      1.147838e+09                5.0              1792147   
249      Wisconsin      2.371696e+09                3.2              5822434   
250        Wyoming      2.313688e+08                3.7               578759   

     SNAPS household_participants  SNAP